实现乘方库函数```pow()```的整数运算。

In [4]:
# 思路：分治法，二分思想，注意有三种情况
# 偶数幂，不断放大x，指数不断减半
# 奇数幂，先将一个x乘到答案中去，然后同偶数幂。该步最多执行两次，首次(奇数幂)跟最后一次(1次方)
# 负指数，返回答案的倒数


def myPow(x: float, n: int) -> float:
    exp = abs(n)
    res = 1
    while exp:
        if exp & 1:
            res *= x
        x *= x
        exp >>= 1

    return res if n >= 0 else 1/res

0.25

In [23]:
# 需要考虑底数为0，指数为0，指数为负数这三种情况
# 当指数为偶数时，可以使用递归求解
# 当指数为奇数时，还要单独乘一次基数


def power(base, exp):
    # 几个基准值
    if base == 0:
        return 0
    if exp == 0:
        return 1
    if exp == 1:
        return base

    neg_flag = 1 if exp < 0 else 0  # 负指数标记
    exp = abs(exp)

    num = power(base, exp >> 1)
    num *= num
    if exp & 0x01 == 1:  # 判断是否奇数
        num *= base

    if neg_flag:
        num = 1 / num

    return num

0.25

实现"."与"*"的正则匹配，前者代表任意字符，后者代表任意次数。

In [11]:
# 注意"*"号的出现是难点，可以匹配0次，可以匹配1次，可以匹配多次
# 注意"*"号的判断优先级要高于"."
def re_match(string, pattern):
    # 几种特殊情况
    if string is None or pattern is None:
        return False
    elif len(string) == 0 and len(pattern) == 0:
        return True
    elif len(string) > 0 and len(pattern) == 0:
        return False

    # 匹配位置后面出现"*"的情况，"*"的优先级要高于"."
    elif len(pattern) > 1 and pattern[1] == '*':
        # 分为两种情况，可以匹配或无法匹配
        if len(string) > 0 and (string[0] == pattern[0] or pattern[0] == '.'):  # 可以匹配，字符相等或出现任意符
            return (re_match(string, pattern[2:]) or  # 跳过'*'
                    re_match(string[1:], pattern[2:]) or  # '*'匹配一次
                    re_match(string[1:], pattern))  # '*'匹配多次
        else:  # 无法匹配
            return re_match(string, pattern[2:])

    # 匹配成功的情况，字符相等或遇到任意匹配符"."
    elif len(string) > 0 and (string[0] == pattern[0] or pattern[0] == '.'):
        return re_match(string[1:], pattern[1:])

    # 字符不等
    else:
        return False

False

In [ ]:
[]()。

判断字符串是否表示一个数值。

In [54]:
# 最简单的，使用内置float()函数判断
# 或者写一个正则匹配规则去判断
# 难点在于模拟正则的匹配过程
# 判断顺序为：[正负号],[整数部分],[.],[小数部分],[e],[符号],[阶码]
# 本体还有一个难点在于Python字符串没有结束符'\0'，所以需要细致考虑越界情况

idx = 0  # 全局指针


def is_numeric(string):
    if string is None or len(string) == 0:
        return False

    global idx
    num_head = scan_int(string)  # 判断是否数字开头，且把指针移至数字后

    # 这一部分代码判断底数的合法性
    if idx < len(string) and string[idx] == '.':
        idx += 1
        # 如果数字打头，则小数点后有无数字无所谓；如果不是数字打头，则小数点后必须有数字。两者中必须有一个成立
        # 注意顺序，继续扫描的优先级要高于后者，否则指针会停在小数点后一位
        num_head = (scan_uint(string) or num_head)

    # 这一部分代码判断阶码的合法性
    if idx < len(string) and (string[idx] == 'e' or string[idx] == 'E'):
        idx += 1
        # 如果数字打头，则e后面必须有数字；如果不是数字打头，那么e后面有数字也没用。两者必须同时成立
        num_head = (num_head and scan_int(string))

    return num_head and idx == len(string)  # 后一个条件用于判断是否扫描完整个字符串


# 扫描数字，并将指针移至数字后
def scan_int(string):
    global idx
    # 有符号则指针后移
    if (idx < len(string)) and (string[idx] == '+' or string[idx] == '-'):
        idx += 1
    return scan_uint(string)


def scan_uint(string):
    global idx
    tmp = idx  # 初始指针位置
    while (idx < len(string)) and (string[idx] >= '0') and (string[idx] <= '9'):
        idx += 1
    return idx > tmp  # 指针变化说明存在数字


is_numeric('123e')

False

实现```int(str)```函数。当str无法转成int类型时返回0。

In [12]:
# 思路：功能实现不难，难点在于各种特殊情况的考虑


def Int(s):
    stat = False    # 数字0与非法字符串都会返回0，需要一个状态变量指示是否非法数字
    if not s or s == '+' or s == '-':
        stat = True
        return 0

    # 提取符号位
    sign = 1
    if s[0] == '+':
        s = s[1:]
    elif s[0] == '-':
        sign = -1
        s = s[1:]

    num = 0
    for ch in s:
        cur_num = ord(ch)-48
        if 0 <= cur_num <= 9:
            num = num*10+cur_num
        else:
            stat = -True
            return 0

    return num*sign


Int('asdas')

0

[Insert Delete GetRandom O(1)](https://leetcode.com/problems/insert-delete-getrandom-o1/)。实现一个高效的随机增删数据结构，还要求能够高效地随机弹出一个元素。

In [ ]:
# 思路：在插入跟删除之前，需要高效地判断该元素是否存在，对应数据结构为dict与set
# 但是该题的题意其实就是考察哈希表的背后实现，准确来说需要一个哈希表与数组
# 元素实际存储在数组中，哈希表只存储元素与数组位置的一个映射关系
# 数组的插入是O(1)，但是删除的话需要把待删除元素与最后一个元素调换再删除才是O(1)
# 至于随机弹出元素，产生随机数即可

import random


class RandomizedSet:

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.stor = list()
        self.table = dict()
        self.ptr = 0    # 指向最后一个元素的指针

    def insert(self, val: int) -> bool:
        """
        Inserts a value to the set. Returns true if the set did not already contain the specified element.
        """
        if val not in self.table.keys():    # 不存在才插入
            self.stor.append(val)
            self.table[val] = self.ptr
            self.ptr += 1
            return True
        return False

    def remove(self, val: int) -> bool:
        """
        Removes a value from the set. Returns true if the set contained the specified element.
        """
        if val in self.table.keys():    # 存在才删除
            item_idx = self.table[val]    # 待删除元素的idx
            last_idx = self.ptr-1     # 末元素的idx
            last_item = self.stor[last_idx]    # 末元素
            self.stor[item_idx] = last_item    # 末元素覆盖掉待删除元素
            self.stor.pop()    # 删除末元素

            # 更新映射表与尾指针
            self.table[last_item] = item_idx
            del self.table[val]
            self.ptr -= 1
            if self.ptr < 0:
                self.ptr = 0
            return True
        return False

    def getRandom(self) -> int:
        """
        Get a random element from the set.
        """
        return self.stor[random.randint(0,self.ptr-1)]